## 모델 구조 변경

개선된 모델(ImprovedMyModel)은 원래 모델(MyModel)에 비해 몇 가지 주요 변경사항을 포함합니다. 이러한 변경사항은 모델의 표현력을 높이고, 과적합을 방지하며, 성능을 개선하는 데 목적이 있습니다. 아래에 개선 과정을 요약해 설명합니다:

1. 추가 컨볼루션 레이어
- 목적: 모델의 학습 능력을 향상시키기 위해 두 번째 컨볼루션 레이어(conv2)를 추가합니다. 이를 통해 모델은 입력 이미지에서 더 복잡하고 추상적인 특징을 추출할 수 있게 됩니다.
- 변경사항: ImprovedMyModel에는 (20, 40, 3) 구성을 가진 두 번째 컨볼루션 레이어가 추가됩니다. 이는 첫 번째 레이어의 출력 채널 수를 입력 채널로 받아, 출력 채널을 40으로 늘리고 커널 크기를 3으로 설정합니다.
2. 드롭아웃 레이어 추가
- 목적: 과적합을 방지하고 일반화 능력을 향상시키기 위해 드롭아웃 레이어를 추가합니다. 드롭아웃은 학습 과정에서 무작위로 일부 뉴런의 출력을 0으로 만들어, 모델이 특정 뉴런에 과도하게 의존하는 것을 방지합니다.
- 변경사항: dropout = nn.Dropout(0.25)를 추가하여, 각 학습 단계에서 뉴런의 25%를 무작위로 비활성화합니다.
3. 완전 연결 레이어의 구조 변경
- 목적: 추가된 컨볼루션 레이어와 드롭아웃 레이어를 통합하고, 모델의 출력 능력을 조정하기 위해 완전 연결 레이어의 구조를 변경합니다.
- 변경사항:
  - 첫 번째 완전 연결 레이어(fc1)의 입력 크기를 조정합니다. 이는 두 번째 컨볼루션 레이어와 맥스풀링 이후의 특징 맵 크기에 기반합니다.
  - 새로운 완전 연결 레이어(fc2)를 추가하여, 출력 크기를 60으로 설정합니다. 이를 통해 모델은 중간 표현을 더 잘 학습할 수 있습니다.
  - 최종 출력 레이어(fc3)는 10개의 클래스에 대한 예측을 출력합니다.
4. 정방향 전파 함수의 수정
- 목적: 모델 아키텍처의 변경사항을 반영하여 정방향 전파 과정을 수정합니다.
- 변경사항: 정방향 전파(forward) 함수는 두 개의 컨볼루션 레이어와 맥스풀링, 드롭아웃, 평탄화, 세 개의 완전 연결 레이어를 순차적으로 적용합니다. 이 과정은 모델이 입력 이미지에서 복잡한 특징을 추출하고, 최종적으로 분류 결정을 내리도록 합니다.

이러한 개선을 통해 ImprovedMyModel은 더 깊은 아키텍처와 과적합 방지 기법을 사용하여 입력 데이터에서 더 정교한 특징을 학습할 수 있으며, 따라서 더 높은 성능을 달성할 가능성이 있습니다.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Q. mnist 데이터 셋에 대해서 Pytorch를 적용하여 모델 구성 변경, 조기 학습 중단을 적용하여 학습하고 Best model을 저장한 후 다시 불러와서 테스트 데이터로 평가한 결과를 출력하세요.

In [2]:
import torch
from torch.utils.data import DataLoader, random_split
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
import numpy as np
import random

# 시드 고정 함수 정의
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # 멀티-GPU를 사용하는 경우
    torch.backends.cudnn.deterministic = True # 모델의 학습과 추론 과정이 완전히 재현 가능하도록 보장
    torch.backends.cudnn.benchmark = False # 동적 벤치마킹 과정을 비활성화하고, 더 일관된 연산 성능을 제공
    torch.use_deterministic_algorithms(True) # 애플리케이션 전반에 걸쳐 결정론적 알고리즘 사용을 강제

# 시드 고정 함수 호출
set_seed(42)

# MNIST 데이터셋을 위한 전처리 과정 정의
# 모델의 학습 효율성을 향상시키고, 일반적으로 신경망에서 더 나은 성능을 얻기 위해 널리 사용
transform = transforms.Compose([
    transforms.ToTensor(),  # 높이, 너비, 채널(예: RGB)의 3차원 배열을 0과 1 사이의 값으로 스케일링된 텐서로 변경
    transforms.Normalize((0.5,), (0.5,))  # 이미지를 평균 0.5, 표준편차 0.5로 정규화하여 [-1, 1] 범위로 조정
])

# MNIST 데이터셋 로드
# transform=transform: 이미지에 적용할 전처리 과정을 지정
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# 훈련 데이터셋을 훈련 및 검증 세트로 분할
train_size = int(0.8 * len(train_dataset))  # 훈련 세트 크기를 전체의 80%로 설정
val_size = len(train_dataset) - train_size  # 검증 세트 크기 계산
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])  # 분할 실행

# DataLoader는 데이터셋에서 미니 배치를 자동으로 생성하고, 이를 모델 학습이나 평가에 이용할 수 있게 해주는 유틸리티
trainloader = DataLoader(train_dataset, batch_size=64, shuffle=True) # 데이터를 로드하기 전에 데이터셋을 무작위로 섞기
valloader = DataLoader(val_dataset, batch_size=64, shuffle=False) # 데이터의 순서가 성능 평가에 영향을 미치지 않는다.
testloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 모델 아키텍처 정의
# ImprovedMyModel 클래스는 1채널 그레이스케일 이미지를 입력으로 받아, 10개의 출력 클래스를 가지는 분류 문제에 사용될 수 있는 구조
class ImprovedMyModel(nn.Module):
    def __init__(self):
        super(ImprovedMyModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5)  # 첫 번째 컨볼루션 레이어 # 커널사이즈는 5by5
        self.pool = nn.MaxPool2d(2, 2)  # 맥스풀링 레이어 # 4개중에 가장큰거 고르고 나머지 삭제하는 맥스풀링..
        self.conv2 = nn.Conv2d(20, 40, 3)  # 두 번째 컨볼루션 레이어 추가 (출력 채널 40, 커널 크기 3)
        self.dropout = nn.Dropout(0.25)  # 드롭아웃 추가 (드롭아웃 확률 0.25)
        self.flatten = nn.Flatten()  # 텐서 평탄화
        # 첫 번째 완전 연결 레이어의 입력 크기를 조정해야 할 수 있음
        self.fc1 = nn.Linear(40 * 5 * 5, 120)  # 입력 크기 조정, 출력 크기를 120으로 변경
        # 40*5*5는 픽셀사이즈가 위에서 평탄화하면서 이렇게 되는데 그과정에서 40*5*5 으로 되는것..
        self.fc2 = nn.Linear(120, 60)  # 새로운 완전 연결 레이어 추가 (출력 크기 60)
        self.fc3 = nn.Linear(60, 10)  # 최종 출력 레이어 (클래스 수 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # 첫 번째 컨볼루션 + 맥스풀링 # 비선형활성화 하고 그것을 풀링까지..
        x = self.pool(F.relu(self.conv2(x)))  # 두 번째 컨볼루션 + 맥스풀링
        x = self.dropout(x)  # 드롭아웃 적용
        x = self.flatten(x)  # 평탄화
        x = F.relu(self.fc1(x))  # 첫 번째 완전 연결 레이어 + ReLU
        x = F.relu(self.fc2(x))  # 두 번째 완전 연결 레이어 + ReLU
        x = self.fc3(x)  # 최종 출력
        return x

model = ImprovedMyModel()  # 모델 인스턴스 생성

# 손실 함수 및 최적화 알고리즘 지정
criterion = nn.CrossEntropyLoss()  # 크로스 엔트로피 손실 함수
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)  # SGD 최적화 알고리즘

# 모델 훈련
best_val_loss = float('inf')  # 검증 손실 값을 무한대로 초기화. 처음에는 모든 검증 손실이 이 값보다 작다고 가정
patience, trials = 5, 0  # 조기 종료 기준 설정
num_epochs = 20  # 에폭 수 설정
for epoch in range(num_epochs):
    model.train()  # 모델을 훈련 모드로 설정
    running_loss = 0.0
    for inputs, labels in trainloader:
        optimizer.zero_grad()  # 그래디언트 초기화
        outputs = model(inputs)  # 모델을 통한 순전파
        loss = criterion(outputs, labels)  # 손실 계산
        loss.backward()  # 역전파
        optimizer.step()  # 파라미터 업데이트
        running_loss += loss.item()

    # 검증 단계
    val_loss = 0.0
    model.eval()  # 모델을 평가 모드로 설정
    with torch.no_grad():
        for inputs, labels in valloader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    val_loss /= len(valloader)  # 평균 검증 손실 계산
    print(f'Epoch {epoch+1}, Train Loss: {running_loss / len(trainloader)}, Val Loss: {val_loss}')

    # 검증 손실이 개선되었는지 확인하고 모델 저장
    if val_loss < best_val_loss:
        print(f'Validation Loss Decreased({best_val_loss:.6f}--->{val_loss:.6f}) \t Saving The Model')
        best_val_loss = val_loss
        trials = 0
        save_path = "/content/drive/MyDrive/hjh_kita_directory/Github/kita_231026/m6_dl/data/model/best_model2.pth"
        torch.save(model.state_dict(), save_path)  # 모델 저장
    else:
        trials += 1
        if trials >= patience:  # 조기 종료 조건 충족 확인
            print("Early stopping triggered")
            break

# 최고의 모델을 불러와서 평가
model.load_state_dict(torch.load(save_path))  # 모델 상태 불러오기

# 모델 평가
correct = 0
total = 0
with torch.no_grad():  # 그래디언트 계산 비활성화
    for inputs, labels in testloader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1) # max_values, max_indices = torch.max(t, dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total  # 정확도 계산
print(f'Accuracy on the 10000 test images: {accuracy}%')  # 정확도 출력

100%|██████████| 9912422/9912422 [00:00<00:00, 94230262.97it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 24506513.01it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 24908577.30it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 37427364.97it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Epoch 1, Train Loss: 0.4320345055907965, Val Loss: 0.0927548280476216
Validation Loss Decreased(inf--->0.092755) 	 Saving The Model
Epoch 2, Train Loss: 0.07796046690115084, Val Loss: 0.056028031976893544
Validation Loss Decreased(0.092755--->0.056028) 	 Saving The Model
Epoch 3, Train Loss: 0.058214919812666875, Val Loss: 0.049590681532169986
Validation Loss Decreased(0.056028--->0.049591) 	 Saving The Model
Epoch 4, Train Loss: 0.04520329272622863, Val Loss: 0.046049785912978126
Validation Loss Decreased(0.049591--->0.046050) 	 Saving The Model
Epoch 5, Train Loss: 0.037527246068464595, Val Loss: 0.038781398403559854
Validation Loss Decreased(0.046050--->0.038781) 	 Saving The Model
Epoch 6, Train Loss: 0.03308063800306991, Val Loss: 0.039987441042575615
Epoch 7, Train Loss: 0.029733163448555085, Val Loss: 0.03948143626095113
Epoch 8, Train Loss: 0.02570551196912614, Val Loss: 0.037455865078571036
Validation L

Q. 저장된 모델 best_model2.pth를 불러와서 epochs 10을 추가하여 학습한 후 test 데이터로 평가를 수행하고 모델을 best_model3.pth로 저장하세요.

In [4]:
import torch
from torch.utils.data import DataLoader, random_split
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
import numpy as np
import random

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends_cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_size = int

SyntaxError: expected ':' (<ipython-input-4-b2cc6186ae63>, line 12)

In [3]:
eimport torch
from torch.utils.data import DataLoader, random_split
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
import numpy as np
import random

# 시드 고정 함수 정의
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # 멀티-GPU를 사용하는 경우
    torch.backends.cudnn.deterministic = True # 모델의 학습과 추론 과정이 완전히 재현 가능하도록 보장
    torch.backends.cudnn.benchmark = False # 동적 벤치마킹 과정을 비활성화하고, 더 일관된 연산 성능을 제공

# 시드 고정 함수 호출
set_seed(42)

# MNIST 데이터셋을 위한 전처리 과정 정의
# 모델의 학습 효율성을 향상시키고, 일반적으로 신경망에서 더 나은 성능을 얻기 위해 널리 사용
transform = transforms.Compose([
    transforms.ToTensor(),  # 높이, 너비, 채널(예: RGB)의 3차원 배열을 0과 1 사이의 값으로 스케일링된 텐서로 변경
    transforms.Normalize((0.5,), (0.5,))  # 이미지를 평균 0.5, 표준편차 0.5로 정규화하여 [-1, 1] 범위로 조정
])

# MNIST 데이터셋 로드
# transform=transform: 이미지에 적용할 전처리 과정을 지정
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# 훈련 데이터셋을 훈련 및 검증 세트로 분할
train_size = int(0.8 * len(train_dataset))  # 훈련 세트 크기를 전체의 80%로 설정
val_size = len(train_dataset) - train_size  # 검증 세트 크기 계산
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])  # 분할 실행

# DataLoader는 데이터셋에서 미니 배치를 자동으로 생성하고, 이를 모델 학습이나 평가에 이용할 수 있게 해주는 유틸리티
trainloader = DataLoader(train_dataset, batch_size=64, shuffle=True) # 데이터를 로드하기 전에 데이터셋을 무작위로 섞기
valloader = DataLoader(val_dataset, batch_size=64, shuffle=False) # 데이터의 순서가 성능 평가에 영향을 미치지 않는다.
testloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 모델 아키텍처 정의
# ImprovedMyModel 클래스는 1채널 그레이스케일 이미지를 입력으로 받아, 10개의 출력 클래스를 가지는 분류 문제에 사용될 수 있는 구조
class ImprovedMyModel(nn.Module):
    def __init__(self):
        super(ImprovedMyModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5)  # 첫 번째 컨볼루션 레이어
        self.pool = nn.MaxPool2d(2, 2)  # 맥스풀링 레이어
        self.conv2 = nn.Conv2d(20, 40, 3)  # 두 번째 컨볼루션 레이어 추가 (출력 채널 40, 커널 크기 3)
        self.dropout = nn.Dropout(0.25)  # 드롭아웃 추가 (드롭아웃 확률 0.25)
        self.flatten = nn.Flatten()  # 텐서 평탄화
        # 첫 번째 완전 연결 레이어의 입력 크기를 조정해야 할 수 있음
        self.fc1 = nn.Linear(40 * 5 * 5, 120)  # 입력 크기 조정, 출력 크기를 120으로 변경
        self.fc2 = nn.Linear(120, 60)  # 새로운 완전 연결 레이어 추가 (출력 크기 60)
        self.fc3 = nn.Linear(60, 10)  # 최종 출력 레이어 (클래스 수 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # 첫 번째 컨볼루션 + 맥스풀링
        x = self.pool(F.relu(self.conv2(x)))  # 두 번째 컨볼루션 + 맥스풀링
        x = self.dropout(x)  # 드롭아웃 적용
        x = self.flatten(x)  # 평탄화
        x = F.relu(self.fc1(x))  # 첫 번째 완전 연결 레이어 + ReLU
        x = F.relu(self.fc2(x))  # 두 번째 완전 연결 레이어 + ReLU
        x = self.fc3(x)  # 최종 출력
        return x

# 모델 아키텍처 인스턴스 생성
model = ImprovedMyModel()

# 저장된 state_dict 불러오기
state_dict = torch.load('/content/drive/MyDrive/hjh_kita_directory/Github/kita_231026/m6_dl/data/model/best_model2.pth')

# 불러온 state_dict를 모델에 로드
model.load_state_dict(state_dict)

# 모델 구조 출력
print(model)

# 손실 함수 및 최적화 알고리즘 지정
criterion = nn.CrossEntropyLoss()  # 크로스 엔트로피 손실 함수
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)  # SGD 최적화 알고리즘

# epochs 추가 학습을 위한 설정
num_additional_epochs = 10
for epoch in range(num_additional_epochs):
    model.train()  # 모델을 훈련 모드로 설정
    running_loss = 0.0
    for inputs, labels in trainloader:
        optimizer.zero_grad()  # 그래디언트 초기화
        outputs = model(inputs)  # 모델을 통한 순전파
        loss = criterion(outputs, labels)  # 손실 계산
        loss.backward()  # 역전파
        optimizer.step()  # 파라미터 업데이트
        running_loss += loss.item()
    # 여기서는 간단화를 위해 검증 단계는 생략했습니다. 필요에 따라 추가하세요.
    print(f'Epoch {epoch+1}, Train Loss: {running_loss / len(trainloader)}')


# 모델 평가
correct = 0
total = 0
with torch.no_grad():  # 그래디언트 계산 비활성화
    for inputs, labels in testloader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total  # 정확도 계산
print(f'Accuracy on the 10000 test images: {accuracy}%')  # 정확도 출력

import os
# Define the save path
save_path = '/content/drive/MyDrive/hjh_kita_directory/Github/kita_231026/m6_dl/data/model/best_model3.pth'

# Ensure directory exists
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# 모델의 매개변수(가중치와 편향)만 저장
torch.save(model.state_dict(), save_path)
print("Saved PyTorch Model State to model.pth")

ImprovedMyModel(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(20, 40, kernel_size=(3, 3), stride=(1, 1))
  (dropout): Dropout(p=0.25, inplace=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=1000, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (fc3): Linear(in_features=60, out_features=10, bias=True)
)
Epoch 1, Train Loss: 0.010379539783309156
Epoch 2, Train Loss: 0.011699398231581047
Epoch 3, Train Loss: 0.00912322826905923
Epoch 4, Train Loss: 0.009984595322600702
Epoch 5, Train Loss: 0.009502588192047067
Epoch 6, Train Loss: 0.008114215796119727
Epoch 7, Train Loss: 0.008531275776411348
Epoch 8, Train Loss: 0.00936103180717934
Epoch 9, Train Loss: 0.00803397601218118
Epoch 10, Train Loss: 0.008708202411192663
Accuracy on the 10000 test images: 99.09%
Saved PyTorch Model State to model.pth
